In [116]:
import pandas as pd
import subprocess
import numpy as np
from math import sin, cos, sqrt, atan2, radians # Para calcular distacias entre 2 ubicaciones (lat, long)
from datetime import datetime #Para categorizar las horas
import pandas_profiling
import matplotlib
import scipy

In [117]:
#Rutas a los csv generados con OpenRefine
_ruta_motivos_viaje='Viajes_bogota_preprocessing-sinpicos.csv' 
_ruta_troncales_transmi = 'troncales.csv'

#Ruta del archivo csv de salida
_ruta_salida_arbol='arbol-preprocessed.csv' 

# Leer el archivo

In [118]:

_df_viajes= pd.read_csv(_ruta_motivos_viaje)
_df_troncales= pd.read_csv(_ruta_troncales_transmi)


_df_viajes.columns = ['motivo viaje','hora inicio', 'hora fin', 'latitud origen', 'latitud destino', 'longitud origen', 
                      'longitud destino', 'duracion viaje', 'dia habil']
_df_troncales.columns = ['codigo troncal', 'longitud', 'latitud']


#Reemplazar las comillas dobles por simples en los nombres. 
#_df['nombre']=_df['nombre'].str.replace('"','-')


# Calcular distancias
Método que calcula las distancias dadas la latitud y la longitud de 2 ubicaciones

In [119]:
def distancia(latitud1, longitud1, latitud2, longitud2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(latitud1)
    lon1 = radians(longitud1)
    lat2 = radians(latitud2)
    lon2 = radians(longitud2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c


# Combinar datasets
Se consolida en un solo CSV las 2 bases de datos a usar

In [120]:
#Motivos de viaje
_latitudes_origen =_df_viajes['latitud origen']
_latitudes_destino =_df_viajes['latitud destino']
_longitudes_origen =_df_viajes['longitud origen']
_longitudes_destino =_df_viajes['longitud destino']

#Troncales transmi
_latitud_troncales = _df_troncales['latitud']
_longitud_troncales = _df_troncales['longitud']
_troncal = _df_troncales['codigo troncal']

In [121]:
def insertarNuevoValor(lat, lon, name):
    troncales = []
    for i in range(len(lat)):
        distancia_minima = -1
        troncal = ""
        
        for y in range(len(_latitud_troncales)):
            dist = distancia(lat[i], lon[i], _latitud_troncales[y], _longitud_troncales[y])
            if distancia_minima == -1 or distancia_minima > dist:
                distancia_minima = dist
                troncal = _troncal[y]
                
        troncales.append(troncal)
    _df_viajes[name] = troncales

In [122]:
insertarNuevoValor(_latitudes_origen, _longitudes_origen, 'troncal origen')
insertarNuevoValor(_latitudes_destino, _longitudes_destino, 'troncal destino')

# Generalizar motivos de viaje

In [123]:
_df_viajes['motivo viaje'].value_counts()

Volver a casa                                     8014
Trabajar                                          3900
Estudiar                                          1686
Tramites                                           808
Recibir atencion en salud                          747
Otra cosa                                          682
Asuntos de Trabajo                                 301
Ir a ver a alguien                                 244
Compras                                            174
Recreacion                                          67
Buscar / Dejar alguien bajo su cuidad               62
Buscar/dejar dejar algo                             58
Buscar trabajo                                      50
Buscar / dejar a alguien que no esta bajo su c      20
Comer / Tomar algo                                   7
Name: motivo viaje, dtype: int64

In [124]:
# Se generaliza asuntos de trabajo
col_loc_motivo = _df_viajes.columns.get_loc('motivo viaje')

indices_asuntos_trabajo = np.concatenate( (np.where(_df_viajes['motivo viaje'] == 'Trabajar')[0], 
                                            np.where(_df_viajes['motivo viaje'] == 'Asuntos de Trabajo')[0],
                                            np.where(_df_viajes['motivo viaje'] == 'Buscar trabajo')[0])
                                        )
for imam in indices_asuntos_trabajo:
    _df_viajes.iloc[imam, col_loc_motivo] = 'Asuntos de trabajo'

# Se generaliza otra cosa (Ir a ver a alguien, Compras, Recreación, Buscar alguien / algo, Comer / Tomar algo)
indices_otra_cosa = np.concatenate( (np.where(_df_viajes['motivo viaje'] == 'Comer / Tomar algo')[0], 
                                     np.where(_df_viajes['motivo viaje'] == 'Recreacion')[0],
                                     np.where(_df_viajes['motivo viaje'] == 'Compras')[0],
                                     np.where(_df_viajes['motivo viaje'] == 'Buscar / Dejar alguien bajo su cuidad')[0],
                                     np.where(_df_viajes['motivo viaje'] == 'Buscar/dejar dejar algo')[0],
                                     np.where(_df_viajes['motivo viaje'] == 'Buscar / dejar a alguien que no esta bajo su c')[0],
                                     np.where(_df_viajes['motivo viaje'] == 'Ir a ver a alguien')[0],
                                     np.where(_df_viajes['motivo viaje'] == 'Otra cosa')[0]) 
                                  )
for imav in indices_otra_cosa:
    _df_viajes.iloc[imav, col_loc_motivo] = 'Otra cosa'
    

In [125]:
_df_viajes['motivo viaje'].value_counts()

Volver a casa                8014
Asuntos de trabajo           4251
Estudiar                     1686
Otra cosa                    1314
Tramites                      808
Recibir atencion en salud     747
Name: motivo viaje, dtype: int64

In [126]:
# Primero se borra la hora final (Mariaaa hay que borrar eso desde el comienzo)
_df_viajes = _df_viajes.drop('hora fin', 1)
_df_viajes = _df_viajes.drop('latitud origen', 1)
_df_viajes = _df_viajes.drop('latitud destino', 1)
_df_viajes = _df_viajes.drop('longitud origen', 1)
_df_viajes = _df_viajes.drop('longitud destino', 1)

# Categorizar según horas

### De lunes a sábado las categorías son las siguientes:

valle 1: 4:00 - 5:59 AM

pico 1: 6:00 - 8:29 

valle 2: 8:30 - 9:29 

pico 2: 9:30 - 15:29

valle 3: 15:30 - 16:29

pico 3: 16:30 - 19:29

valle 4: 19:30 - fin de operación 


### Para domingos y festivos:

valle dyf: Todo el día


In [127]:
horas_limite = {
    'valle_1': '04:00:00',
    'pico_1': '06:00:00',
    'valle_2': '08:30:00',
    'pico_2': '09:30:00',
    'valle_3': '15:30:00',
    'pico_3': '16:30:00',
    'valle_4': '19:30:00',
    'fin_dia': '00:00:00'
}

In [128]:
formato_hora_limite = '%H:%M:%S'    

def categoria_por_hora(hora, dia):
    valle_1 = datetime.strptime( horas_limite['valle_1'], formato_hora_limite).time()
    pico_1 = datetime.strptime( horas_limite['pico_1'], formato_hora_limite).time()
    valle_2 = datetime.strptime( horas_limite['valle_2'], formato_hora_limite).time()
    pico_2 = datetime.strptime( horas_limite['pico_2'], formato_hora_limite).time()
    valle_3 = datetime.strptime( horas_limite['valle_3'], formato_hora_limite).time()
    pico_3 = datetime.strptime( horas_limite['pico_3'], formato_hora_limite).time()
    valle_4 = datetime.strptime( horas_limite['valle_4'], formato_hora_limite).time()
    fin_dia = datetime.strptime( horas_limite['fin_dia'], formato_hora_limite).time()
    
    categoria = 'no-definida'
    if dia == 0: 
        categoria = 'valle-dyf'
    else:
        if hora >= valle_1 and hora < pico_1:
            categoria = 'Valle-1'
        elif hora >= pico_1 and hora < valle_2:
            categoria = 'Pico-1'
        elif hora >= valle_2 and hora < pico_2:
            categoria = 'Valle-2'
        elif hora >= pico_2 and hora < valle_3:
            categoria = 'Pico-2'
        elif hora >= valle_3 and hora < pico_3:
            categoria = 'Valle-3'
        elif hora >= pico_3 and hora < valle_4:
            categoria = 'Pico-3'
        elif hora >= valle_4 and hora < fin_dia:
            categoria = 'Valle-4'
    
    return categoria

In [129]:
_horas_iniciales =_df_viajes['hora inicio']
_dia_habil = _df_viajes['dia habil']
categorias_horas_iniciales = []

for i in range(len(_horas_iniciales)):
    hora_final = datetime.strptime(_horas_iniciales[i], formato_hora_limite).time()
    categorias_horas_iniciales.append(categoria_por_hora(hora_final, _dia_habil[i]))

_df_viajes['categoria_hora'] = categorias_horas_iniciales

In [130]:
#Borro columnas que ya no necesito
_df_viajes = _df_viajes.drop('hora inicio', 1)

In [132]:
_df_viajes

,motivo viaje,duracion viaje,dia habil,troncal origen,troncal destino,categoria_hora
0,Asuntos de trabajo,00:33:00,1,G,A,Pico-3
1,Volver a casa,00:47:00,1,A,G,no-definida
2,Asuntos de trabajo,02:00:00,0,L,C,valle-dyf
3,Volver a casa,01:10:00,0,C,F,valle-dyf
4,Asuntos de trabajo,01:10:00,0,F,B,valle-dyf
5,Volver a casa,01:10:10,0,B,F,valle-dyf
6,Estudiar,01:20:00,0,F,E,valle-dyf
7,Volver a casa,01:20:00,0,E,F,valle-dyf
8,Estudiar,00:46:00,0,F,A,valle-dyf
9,Volver a casa,01:00:00,0,A,F,valle-dyf


# Categorizar duración de viaje

Corto: 00:30:00 o menos

Medio: 00:31:00 - 01:30:00

Largo: 01:31:00 - 02:30:00

Muy largo: 02:31:00 o más 



In [133]:
horas_fin_categorias = {
    'corto': '00:30:00',
    'medio': '01:30:00',
    'largo': '02:30:00',
}

In [134]:
formato_hora_limite = '%H:%M:%S'    

def categoria_por_tiempo_viaje(hora):
    corto = datetime.strptime( horas_fin_categorias['corto'], formato_hora_limite).time()
    medio = datetime.strptime( horas_fin_categorias['medio'], formato_hora_limite).time()
    largo = datetime.strptime( horas_fin_categorias['largo'], formato_hora_limite).time()
    
    if hora <= corto:
        categoria = 'Corto'
    elif hora <= medio:
        categoria = 'Medio'
    elif hora <= largo:
        categoria = 'Largo'
    else:
        categoria = 'Muy Largo'
    
    return categoria

In [135]:
_duracion =_df_viajes['duracion viaje']
categorias_tiempo_viajes = []

for hora in _duracion:
    hora_final = datetime.strptime(hora, formato_hora_limite).time()
    categorias_tiempo_viajes.append(categoria_por_tiempo_viaje(hora_final))

_df_viajes['categoria viaje'] = categorias_tiempo_viajes

In [136]:
#Elimino todas las filas que tengan la categoria de viaje como no-definida, ya que esto representa una inconsistencia 
#en los datos
_df_viajes = _df_viajes[_df_viajes.categoria_hora != 'no-definida']
#Borro columnas que ya no necesito
_df_viajes = _df_viajes.drop('duracion viaje', 1)

# Exportar como CSV - Árbol

Se exporta un CSV el cual se usará como input para clasificar en un árbol de decisión 

In [137]:
#Exportar a csv
_df_viajes.to_csv(_ruta_salida_arbol, index=False, encoding='utf8' )

# Pandas Profiling

In [82]:
pandas_profiling.ProfileReport(_df_viajes)

Number of variables,7
Number of observations,15694
Total Missing (%),0.0%
Total size in memory,858.3 KiB
Average record size in memory,56.0 B
Numeric,2
Categorical,5
Date,0
Text (Unique),0
Rejected,0
Distinct count,4


# Binarizar los datos

In [112]:
_df_viajes['motivo viaje'].value_counts()

Volver a casa                7006
Asuntos de trabajo           4230
Estudiar                     1669
Otra cosa                    1252
Tramites                      797
Recibir atencion en salud     740
Name: motivo viaje, dtype: int64

In [1]:
_grouped =_df_viajes.groupby(_df_viajes.index)['troncal origen'].apply(lambda x: '|'.join(x)).reset_index() 


NameError: name '_df_viajes' is not defined

In [83]:
_grouped

,index,0
0,0,troncal origen|troncal destino
1,2,troncal origen|troncal destino
2,3,troncal origen|troncal destino
3,4,troncal origen|troncal destino
4,5,troncal origen|troncal destino
5,6,troncal origen|troncal destino
6,7,troncal origen|troncal destino
7,8,troncal origen|troncal destino
8,9,troncal origen|troncal destino
9,10,troncal origen|troncal destino


# Obtener una columna por cada autor

In [77]:
#Transforma el dataset para obtener una columna binaria por cada autor. 
#Esto puede tardar bastante tiempo, dependiendo del tamaño del dataset.
_dummies=_grouped['entrevistado'].str.get_dummies('|')

KeyError: 'entrevistado'

In [11]:
#Agregar al dataset las columnas agrupadas
_df=pd.concat([_grouped, _dummies],axis=1)